In [1]:
nn = require("nn")
nninit = require("nninit")
require 'optim'
csvigo =  require("csvigo")
require 'torch'

Read data

In [2]:
-- load trainning_size
matrix_words = csvigo.load{path='pre-processing/train_data/good_train_matrix_file.txt', mode='large', separator=' '}
classes = {"O", "B-LOC", "B-PER", "B-ORG", "B-TOUR", "I-ORG", "I-PER", "I-TOUR", "I-LOC", "B-PRO", "I-PRO"}

windowSize = #matrix_words[1]
trainingSize = #matrix_words

data = torch.Tensor(trainingSize, windowSize)
for i=1, trainingSize do
    for j=1, windowSize do
        data[i][j] = tonumber(matrix_words[i][j]) + 1 -- lua index start from 1, not 0
    end
end


-- read labels
labelsRaw = csvigo.load{path='pre-processing/train_data/good_train_label_file.txt', mode='large', separator=' '}
labels = torch.DoubleTensor(#labelsRaw)
for i=1, #labelsRaw do    
    labels[i] = tonumber(labelsRaw[i][1]) + 1
end

print(string.format("window_size = %d, training_size = %d", windowSize, trainingSize))

<csv>	parsing file: pre-processing/train_data/good_train_matrix_file.txt	


<csv>	parsing done	


<csv>	parsing file: pre-processing/train_data/good_train_label_file.txt	


<csv>	parsing done	


In [3]:
-- load test_data -- 
-- load trainning_size
matrix_words_test = csvigo.load{path='pre-processing/train_data/good_test_matrix_file.txt', mode='large', separator=' '}
testingSize = #matrix_words_test

data_test = torch.Tensor(testingSize, windowSize)
for i=1, testingSize do
    for j=1, windowSize do        
        data_test[i][j] = tonumber(matrix_words_test[i][j]) + 1 -- lua index start from 1, not 0
    end
end


-- read labels -- 
labelsRawTest = csvigo.load{path='pre-processing/train_data/good_test_label_file.txt', mode='large', separator=' '}
labels_test = torch.DoubleTensor(#labelsRawTest)
for i=1, #labelsRawTest do    
     labels_test[i] = tonumber(labelsRawTest[i][1]) + 1
end

print(string.format("window_size = %d, testing_size = %d", windowSize, testingSize))

<csv>	parsing file: pre-processing/train_data/good_test_matrix_file.txt	


<csv>	parsing done	


<csv>	parsing file: pre-processing/train_data/good_test_label_file.txt	


<csv>	parsing done	


window_size = 11, testing_size = 333446	


Config some parameters

In [6]:
-- load data 
w2v_mat = csvigo.load{path='pre-processing/train_data/total_matrix_final.txt', mode='large', separator=' '}

rows = #w2v_mat
cols = #w2v_mat[1] - 1 -- the last elem is \n

-- load word
words = csvigo.load{path='pre-processing/train_data/total_words_final.txt', mode='large', separator=' '}
dictSize = #words
embeddedSize = cols

print (string.format("Size {rows=%d, cols=%d}", rows, cols))
print (string.format("Number of total words = %d", dictSize))

Generate new tensor at size 16458 x 200	


<csv>	parsing file: pre-processing/train_data/total_matrix_final.txt	


<csv>	parsing done	
<csv>	parsing file: pre-processing/train_data/total_words_final.txt	


In [7]:
csv_tensor = torch.Tensor(rows, cols)
print(string.format("Generate new tensor at size %d x %d", rows, cols))

Generate new tensor at size 16458 x 200	


Load csv data to a Tensor

In [8]:
for i=1, rows do
    for j=1, cols do                 
        csv_tensor[i][j] = tonumber(w2v_mat[i][j])
    end
end


Create lookup table

In [9]:
lookup = nn.LookupTable(dictSize, embeddedSize)
lookup.weight = csv_tensor

print(#lookup:forward(data[1]))

  11
 200
[torch.LongStorage of size 2]



Create Model

In [10]:
-- some matrix layer --
L = 100
K = embeddedSize

windowSize = 11
model = nn.Sequential()
model:add(lookup)
model:add(nn.Reshape(K*windowSize))
model:add(nn.Linear(K*windowSize, L))
model:add(nn.HardTanh())
model:add(nn.Linear(L, #classes))
model:add(nn.SoftMax())

print(model)


nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> output]
  (1): nn.LookupTable
  (2): nn.Reshape(2200)
  (3): nn.Linear(2200 -> 100)
  (4): nn.HardTanh
  (5): nn.Linear(100 -> 11)
  (6): nn.SoftMax
}
{
  gradInput : DoubleTensor - empty
  modules : 


    {
      1 : 
        nn.LookupTable
        {
          output : DoubleTensor - size: 11x200
          gradInput : DoubleTensor - empty
          copiedInput : true
          weight : DoubleTensor - size: 16458x200
          _type : torch.DoubleTensor
          _input : LongTensor - size: 11
          paddingValue : 0
          _count : IntTensor - empty
          shouldScaleGradByFreq : false
          gradWeight : DoubleTensor - size: 16458x200
        }
      2 : 
        nn.Reshape(2200)
        {
          _type : torch.DoubleTensor
      

    output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          size : LongStorage - size: 1
          nelement : 2200
          batchsize : LongStorage - size: 2
        }
      3 : 
        nn.Linear(2200 -> 100)
        {
          gradBias : DoubleTensor - size: 100
          weight : DoubleTensor - size: 100x2200
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          bias : DoubleTensor - size: 100
          gradWeight : DoubleTensor - size: 100x2200
        }
      4 : 
        nn.HardTanh
        {


          inplace : false
          min_val : -1
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          max_val : 1
          _type : torch.DoubleTensor
        }
      5 : 
        nn.Linear(100 -> 11)
        {
          gradBias : DoubleTensor - size: 11
          weight : DoubleTensor - size: 11x100
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          bias : DoubleTensor - size: 11
          gradWeight : DoubleTensor - size: 11x100
        }
      6 : 
        

nn.SoftMax
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
        }
    }
  _type : torch.DoubleTensor
  output : DoubleTensor - empty
}


In [11]:
criterion = nn.ClassNLLCriterion()
x, dl_dx = model:getParameters()

In [12]:
sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}

In [21]:
print(trainingSize)
step = function(batch_size)
    local current_loss = 0
    local count = 0
    local shuffle = torch.randperm(trainingSize)    
    batch_size = batch_size or 200
    
    for t = 0, trainingSize, batch_size do
        if t>=trainingSize then
            break
        end
        
        -- setup inputs and targets for this mini-batch
        local size = math.min(t + batch_size, trainingSize) - t        
        local inputs = torch.Tensor(size, windowSize)
        local targets = torch.Tensor(size)
        for i = 1, size do            
            local input = data[shuffle[i+t]]
            local target = labels[shuffle[i+t]]            
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent            
            outputs = model:forward(inputs);            
            local loss = criterion:forward(model:forward(inputs), targets)        
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]        
   end

    -- normalize loss
    return current_loss / count
end

11456	


In [30]:
print(testingSize)
eval = function(batch_size)
    local count = 0
    batch_size = batch_size or 200        
    
    true_prob = {}
    false_prob = {}
    
    -- init true and false count --
    for i=1, #classes do
        true_prob[i] = 0
        false_prob[i] = 0
    end
    
    for i = 0, testingSize, batch_size do                
        if i >= testingSize then
            break
        end
        
        local size = math.min(i + batch_size, testingSize) - i        
        local inputs = data_test[{{i+1,i+size}}]
        local targets = labels_test[{{i+1,i+size}}]:long()
        local outputs = model:forward(inputs)    
        local _, indices = torch.max(outputs, 2)                
        
        print(indices)
        print(targets)        
        guessed_right = 0                
        for j=1, indices:size()[1] do
            label = targets[j]
            print(string.format("Index %d", j))
            print(indices[j][1])
            print(targets[j])
            print(guessed_right)
            if indices[j][1] == targets[j] then
                guessed_right = guessed_right + 1                             
                true_prob[label] = true_prob[label] + 1
            else 
                false_prob[label] = false_prob[label] + 1
            end
        end                
        
        count = count + guessed_right    
        
        if i > 300 then
            break
        end
    end
    
    return count/testingSize, true_prob, false_prob
end

333446	


In [31]:
max_iters = 1
do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    for iter = 1, max_iters do
        local loss = step(20)        
        print(string.format('Epoch: %d Current loss: %4f', iter, loss))
        local accuracy, true_prob, false_prob = eval(20)        
        print(string.format('Accuracy on the validation set: %4f', accuracy))
        for x, y in pairs(true_prob) do
            print(string.format('Count label %d number of true=%d, number of false=%d, accuracy=%4f', x, y, false_prob[x], y/(y+false_prob[x])))
        end
        
        if accuracy < last_accuracy then
             if decreasing > threshold then break end
             decreasing = decreasing + 1
        else
             decreasing = 0
        end
        last_accuracy = accuracy
    end
end

Epoch: 1 Current loss: -0.388161	


 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 10
 11
 11
 11
 11
 11
 10
[torch.LongTensor of size 20x1]

 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
[torch.LongTensor of size 20]

Index 1	
11	
11	
0	
Index 2	


11	
11	
1	
Index 3	
11	
11	
2	
Index 4	
11	
11	
3	
Index 5	
11	
11	
4	
Index 6	
11	
11	
5	
Index 7	
11	
11	
6	
Index 8	
11	
11	
7	
Index 9	
11	
11	
8	
Index 10	
11	
11	
9	
Index 11	
11	
11	
10	
Index 12	
11	
11	
11	
Index 13	
11	
11	
12	
Index 14	
10	
11	
13	
Index 15	
11	
11	
13	
Index 16	
11	
11	
14	
Index 17	
11	
11	
15	
Index 18	
11	
11	
16	
Index 19	
11	
11	
17	
Index 20	
10	
11	
18	


 11
 11
 10
 11
  2
 11
 10
 11
 11
 11
 10
 11
 11
 10
 11
 11
 11
 11
 11
 11
[torch.LongTensor of size 20x1]

 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
[torch.LongTensor of size 20]

Index 1	
11	
11	
0	
Index 2	
11	
11	
1	
Index 3	
10	
11	
2	
Index 4	
11	
11	
2	
Index 5	
2	
11	
3	
Index 6	
11	
11	
3	
Index 7	
10	
11	
4	
Index 8	
11	
11	
4	
Index 9	
11	
11	
5	
Index 10	
11	
11	
6	
Index 11	
10	
11	
7	
Index 12	
11	
11	
7	


Index 13	
11	
11	
8	
Index 14	
10	
11	
9	
Index 15	
11	
11	
9	
Index 16	
11	
11	
10	
Index 17	
11	
11	
11	
Index 18	
11	
11	
12	
Index 19	
11	
11	
13	
Index 20	
11	
11	
14	


...

 11
 11
  2
  3
  2
  2
  3
  2
  3
  3
  2
  3
  6
  6
  3
  3
  3
  2
  3
  3
[torch.LongTensor of size 20x1]

 11
 11
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
[torch.LongTensor of size 20]

Index 1	
11	
11	
0	
Index 2	
11	
11	
1	
Index 3	
2	
2	
2	
Index 4	
3	
2	
3	
Index 5	
2	
2	
3	
Index 6	
2	
2	
4	
Index 7	
3	
2	
5	
Index 8	
2	
2	
5	
Index 9	
3	
2	
6	
Index 10	
3	
2	
6	
Index 11	
2	


2	
6	
Index 12	
3	
2	
7	


Index 13	
6	
2	
7	
Index 14	
6	
2	
7	
Index 15	
3	
2	
7	
Index 16	


3	
2	
7	
Index 17	
3	
2	
7	
Index 18	
2	
2	
7	
Index 19	
3	
2	
8	
Index 20	
3	
2	


8	
Accuracy on the validation set: 0.000804	
Count label 1 number of true=0, number of false=0, accuracy= nan	
Count label 2 number of true=6, number of false=12, accuracy=0.333333	
Count label 3 number of true=0, number of false=0, accuracy= nan	
Count label 4 number of true=0, number of false=0, accuracy= nan	
Count label 5 number of true=0, number of false=0, accuracy= nan	
Count label 6 number of true=0, number of false=0, accuracy= nan	
Count label 7 number of true=0, number of false=0, accuracy= nan	
Count label 8 number of true=0, number of false=0, accuracy= nan	
Count label 9 number of true=0, number of false=0, accuracy= nan	
Count label 10 number of true=0, number of false=0, accuracy= nan	
Count label 11 number of true=262, number of false=60, accuracy=0.813665	


In [9]:
true_prob = torch.Tensor(#classes)
print(true_prob)
prob = {}
prob[1] = 2
prob[1] ++
print(prob[2] == nil)
print(prob)

[string "true_prob = torch.Tensor(#classes)..."]:5: '=' expected near '+': 

In [19]:
true_prob = {}
false_prob = {}
    
    -- init -- 
    for i=1, #classes do
        true_prob[i] = 0
        false_prob[i] = 0
    end

for x, y in pairs(true_prob) do
    print(x)
    print(y)
end

1	
0	
2	
0	
3	
0	
4	
0	
5	
0	
6	
0	
7	
0	
8	
0	
9	
0	
10	
0	
11	
0	


In [4]:
 a = function(x, y)
        return x, y
    end

m, n = a(1, 2)
print(m)
print(n)
a = torch.Tensor(1, 20)
print(a:size())
print(a[1])

1	
2	
  1
 20
[torch.LongStorage of size 2]

 6.9233e-310
 6.9233e-310
  0.0000e+00
  0.0000e+00
 3.2683e-258
 1.1286e+277
 2.8751e+161
 1.4776e+248
  1.1610e-28
 6.2499e-143
  1.1610e-28
  4.7539e-38
  2.2637e-76
 5.0116e+217
 8.3717e-144
  1.6684e-47
 1.1619e+165
  5.4483e-90
  4.4680e-62
 3.9706e+246
[torch.DoubleTensor of size 20]



In [12]:
inputs = labels_test[{{10,30}}]

In [22]:
print (inputs[1])

11	
